In [1]:
import zipfile
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split
import shutil

In [2]:
!git clone https://github.com/avaneeshp23/AVHackathon.git

Cloning into 'AVHackathon'...
remote: Enumerating objects: 2255, done.
remote: Counting objects: 100% (2255/2255), done.
remote: Compressing objects: 100% (2252/2252), done.
remote: Total 2255 (delta 18), reused 2233 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (2255/2255), 29.36 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [3]:
train_df = pd.read_csv('/content/AVHackathon/train/train.csv')
train_df.head()
train_df.shape

(1646, 2)

In [ ]:
train_df.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [4]:
test_df = pd.read_csv('/content/AVHackathon/test.csv')
test_df.shape

(706, 1)

code to delete folders

In [ ]:
# folder_path = '/content/train_images/non_emergency'

# # Check if the folder exists
# if os.path.exists(folder_path):
#     # Iterate over all the files in the folder
#     for filename in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, filename)
#         try:
#             # Check if it's a file and delete it
#             if os.path.isfile(file_path) or os.path.islink(file_path):
#                 os.unlink(file_path)
#             # Check if it's a directory and delete it
#             elif os.path.isdir(file_path):
#                 shutil.rmtree(file_path)
#         except Exception as e:
#             print(f'Failed to delete {file_path}. Reason: {e}')
# else:
#     print(f'The folder {folder_path} does not exist.')

In [5]:
os.makedirs('train_images', exist_ok=True)
os.makedirs('test_images', exist_ok=True)

In [6]:
def copy_images(df, src_folder, dest_folder):
    for index, row in df.iterrows():
        image_name = row['image_names']
        src_path = os.path.join(src_folder, image_name)
        dest_path = os.path.join(dest_folder, image_name)
        shutil.copy(src_path, dest_path)

In [ ]:
#move_images(train_df, '/content/AVHackathon/train/images', 'train_images')

In [7]:
os.makedirs('train_images/1', exist_ok=True)
os.makedirs('train_images/0', exist_ok=True)

for index, row in train_df.iterrows():
    image_name = row['image_names']
    emergency_or_not = row['emergency_or_not']

    # Define source and destination paths
    src_path = os.path.join('/content/AVHackathon/train/images', image_name)
    if emergency_or_not == 1:
        dest_path = os.path.join('train_images/1', image_name)
    else:
        dest_path = os.path.join('train_images/0', image_name)

    # Copy the image to the respective folder
    shutil.copy(src_path, dest_path)

In [8]:
os.makedirs('test_images', exist_ok=True)
copy_images(test_df, '/content/AVHackathon/train/images', 'test_images')

In [9]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet mean/std
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")

GPU is available


In [11]:
checkpoints_dir = 'train_images/.ipynb_checkpoints'
if os.path.exists(checkpoints_dir):
    shutil.rmtree(checkpoints_dir)
train_dataset = datasets.ImageFolder(root='train_images', transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Hyperparameters to tune
learning_rates = [0.001, 0.01]
batch_sizes = [16, 32]
dropout_rates = [0.1, 0.3, 0.5]
hidden_layers = [1, 2, 3]  # 1 hidden layer or 2 hidden layers
hidden_units = [128, 256]  # Size of each hidden layer
num_epochs = 10

# Placeholder for the best model and hyperparameters
best_model = None
best_accuracy = 0
best_params = {}

# Grid search loop over hyperparameters
for lr in learning_rates:
    for batch_size in batch_sizes:
        for dropout_rate in dropout_rates:
            for num_hidden_layers in hidden_layers:
                for hidden_units_per_layer in hidden_units:

                    # Recreate data loaders with the current batch size
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

                    # Define the CNN model architecture with the current number of hidden layers and dropout rate
                    conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
                    conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
                    conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
                    pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
                    fc_layers = []

                    # Create hidden layers based on the number of hidden layers
                    input_size = 128 * 16 * 16  # Image size after convolution and pooling
                    for i in range(num_hidden_layers):
                        fc_layers.append(nn.Linear(input_size, hidden_units_per_layer))
                        fc_layers.append(nn.ReLU())
                        fc_layers.append(nn.Dropout(dropout_rate))
                        input_size = hidden_units_per_layer  # Update input size for the next layer

                    fc_layers.append(nn.Linear(input_size, 1))  # Output layer for binary classification (0 or 1)

                    # Combine layers into a model
                    model = nn.Sequential(
                        conv1,
                        nn.ReLU(),
                        pool,
                        conv2,
                        nn.ReLU(),
                        pool,
                        conv3,
                        nn.ReLU(),
                        pool,
                        nn.Flatten(),
                        *fc_layers
                    ).to(device)

                    # Define the loss function and optimizer for this combination of hyperparameters
                    criterion = nn.BCEWithLogitsLoss()
                    optimizer = optim.Adam(model.parameters(), lr=lr)

                    # Train the model
                    for epoch in range(num_epochs):
                        model.train()
                        running_loss = 0.0
                        correct = 0
                        total = 0

                        for inputs, labels in train_loader:
                            inputs, labels = inputs.to(device), labels.to(device)

                            optimizer.zero_grad()
                            outputs = model(inputs)
                            loss = criterion(outputs, labels.float().unsqueeze(1))
                            loss.backward()
                            optimizer.step()

                            running_loss += loss.item()
                            predicted = (torch.sigmoid(outputs) > 0.5).float()
                            total += labels.size(0)
                            correct += (predicted.squeeze() == labels).sum().item()

                        epoch_loss = running_loss / len(train_loader)
                        epoch_accuracy = 100 * correct / total
                        print(f"Epoch [{epoch+1}/{num_epochs}] for lr={lr}, batch_size={batch_size}, dropout={dropout_rate}, hidden_layers={num_hidden_layers}, hidden_units={hidden_units_per_layer}: Loss={epoch_loss:.4f}, Accuracy={epoch_accuracy:.2f}%")

                        # Save the best model
                        if epoch_accuracy > best_accuracy:
                            best_accuracy = epoch_accuracy
                            best_model = model
                            best_params = {
                                'learning_rate': lr,
                                'batch_size': batch_size,
                                'dropout_rate': dropout_rate,
                                'num_hidden_layers': num_hidden_layers,
                                'hidden_units_per_layer': hidden_units_per_layer
                            }

print(f"Best Accuracy: {best_accuracy:.2f}% with parameters: {best_params}")


Epoch [1/10] for lr=0.001, batch_size=16, dropout=0.1, hidden_layers=1, hidden_units=128: Loss=0.5996, Accuracy=68.41%
Epoch [2/10] for lr=0.001, batch_size=16, dropout=0.1, hidden_layers=1, hidden_units=128: Loss=0.4890, Accuracy=76.55%
Epoch [3/10] for lr=0.001, batch_size=16, dropout=0.1, hidden_layers=1, hidden_units=128: Loss=0.4254, Accuracy=80.86%
Epoch [4/10] for lr=0.001, batch_size=16, dropout=0.1, hidden_layers=1, hidden_units=128: Loss=0.3516, Accuracy=85.05%
Epoch [5/10] for lr=0.001, batch_size=16, dropout=0.1, hidden_layers=1, hidden_units=128: Loss=0.2988, Accuracy=86.88%
Epoch [6/10] for lr=0.001, batch_size=16, dropout=0.1, hidden_layers=1, hidden_units=128: Loss=0.2044, Accuracy=91.62%
Epoch [7/10] for lr=0.001, batch_size=16, dropout=0.1, hidden_layers=1, hidden_units=128: Loss=0.1330, Accuracy=95.26%
Epoch [8/10] for lr=0.001, batch_size=16, dropout=0.1, hidden_layers=1, hidden_units=128: Loss=0.0689, Accuracy=97.87%
Epoch [9/10] for lr=0.001, batch_size=16, dropou

In [14]:
model = best_model
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

test_images_folder = '/content/test_images'
result_df = pd.DataFrame(columns=['image_names', 'emergency_or_not'])

for image_name in os.listdir(test_images_folder):
    image_path = os.path.join(test_images_folder, image_name)

    # Open the image
    image = Image.open(image_path).convert('RGB')

    # Apply transformations
    image = test_transforms(image)

    # Add batch dimension
    image = image.unsqueeze(0).to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(image)
        predicted = (torch.sigmoid(outputs) > 0.5).float()

    # Append the result to the DataFrame
    result_df = pd.concat([result_df, pd.DataFrame({'image_names': [image_name], 'emergency_or_not': [predicted.item()]})], ignore_index=True)



<ipython-input-14-e6431a4ea8cd>:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({'image_names': [image_name], 'emergency_or_not': [predicted.item()]})], ignore_index=True)


In [15]:
test_csv_path = '/content/AVHackathon/test.csv'
test_df = pd.read_csv(test_csv_path)

In [20]:
if 'emergency_or_not' not in test_df.columns:
    test_df['emergency_or_not'] = None

In [21]:
result_df.head()

,image_names,emergency_or_not
0,1993.jpg,0.0
1,2271.jpg,0.0
2,821.jpg,1.0
3,440.jpg,1.0
4,2175.jpg,0.0


In [17]:
for index, row in result_df.iterrows():
    test_df.loc[test_df['image_names'] == row['image_names'], 'emergency_or_not'] = row['emergency_or_not']

In [23]:
test_df.to_csv(test_csv_path, index=False)

In [24]:
test_df.head()

,image_names,emergency_or_not
0,1960.jpg,0.0
1,668.jpg,1.0
2,2082.jpg,0.0
3,808.jpg,1.0
4,1907.jpg,0.0


In [25]:
# prompt: updated_test_df  = pd.read_csv(test_csv_path)

updated_test_df  = pd.read_csv(test_csv_path)
updated_test_df.head()

,image_names,emergency_or_not
0,1960.jpg,0.0
1,668.jpg,1.0
2,2082.jpg,0.0
3,808.jpg,1.0
4,1907.jpg,0.0
